In [53]:
from typing_extensions import TypedDict
from typing import Annotated
from langchain.chat_models import init_chat_model
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph,START,END


In [54]:
llm = init_chat_model("google_genai:gemini-2.0-flash")





In [55]:
llm.invoke("Hello, world!")



AIMessage(content='Hello there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--9112e4b6-bfd2-4ad1-9e99-be062945c18c-0', usage_metadata={'input_tokens': 4, 'output_tokens': 11, 'total_tokens': 15, 'input_token_details': {'cache_read': 0}})

In [56]:
class State(TypedDict):
    messages:Annotated[list,add_messages]


In [57]:
def chatbot(state:State)->State:
    return {"messages":[llm.invoke(state["messages"])]}

In [58]:
builder = StateGraph(State)


In [59]:
builder.add_node("chatbot",chatbot)
builder.add_edge(START,"chatbot")
builder.add_edge("chatbot",END)
graph = builder.compile()

In [62]:
state = None
while True:
    in_massage = input("you:")
    if in_massage.lower() == "end":
        break
    if state is None:
        state:State = {
            "messages":[{
                "role":"user","content":in_massage
            }]
        }
    else :
        state["messages"].append({"role":"user","content":in_massage})
    state = graph.invoke(state)
    print("me:",state["messages"][-2].content)
    print("bot:",state["messages"][-1].content)

me: hi
bot: Hi there! How can I help you today?


KeyboardInterrupt: Interrupted by user